In [42]:
import time
import sklearn
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from baggingPU import BaggingClassifierPU

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

In [43]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

# import data

In [44]:
#df_raw = pd.read_csv('../data/w-dependence.csv')

#df_raw = pd.read_csv('../data/1place-dependence.csv')

df_raw = pd.read_csv('../data/w-related.csv')


df_raw['label'] = df_raw['label'].astype("int")
print(df_raw.label.value_counts())
print('Has null values', df_raw.isnull().values.any())

1    10000
0    10000
Name: label, dtype: int64
Has null values False


In [45]:
df_raw.head(10)

,p1,p2,p3,p4,p5,p6,p7,p8,label
0,0,1,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,1
2,0,1,1,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,1
4,1,0,1,0,0,0,0,0,1
5,0,0,0,0,1,1,0,0,1
6,0,0,0,0,0,0,1,0,1
7,0,1,2,0,0,0,0,0,1
8,0,0,1,1,0,0,0,0,1
9,0,0,0,0,0,0,0,1,1


In [46]:
print(df_raw.iloc[:10000,:])
df_max = np.amax(df_raw.iloc[:10000,[2,5]])
print(df_max)

      p1  p2  p3  p4  p5  p6  p7  p8  label
0      0   1   0   0   0   0   0   0      1
1      1   0   0   0   0   0   0   0      1
2      0   1   1   0   0   0   0   0      1
3      0   0   0   1   0   0   0   0      1
4      1   0   1   0   0   0   0   0      1
...   ..  ..  ..  ..  ..  ..  ..  ..    ...
9995   0   0   6   0   1  38   0   0      1
9996   0   0   6   0   0  37   1   0      1
9997   1   0   6   0   0  37   0   0      1
9998   0   0   4   0   1  39   0   0      1
9999   0   0   4   0   0  38   1   0      1

[10000 rows x 9 columns]
p3    81
p6    40
dtype: int64


In [47]:
print(df_raw.iloc[10000:,:])
print(np.amax(df_raw.iloc[10000:,[2,5]]))

       p1  p2  p3  p4  p5  p6  p7  p8  label
10000   1   0  62   0   1  81   0   0      0
10001   1   1  52   1   1  18   0   0      0
10002   1   1  71   0   1  20   0   0      0
10003   1   1  37   0   1  61   1   1      0
10004   1   0  47   1   0  12   0   1      0
...    ..  ..  ..  ..  ..  ..  ..  ..    ...
19995   9   8  28   0   4  91   2   7      0
19996   1   9  98   2   9  85   9   7      0
19997   0   8  49   3   1  17   6   4      0
19998   1   2  35   9   7  55   3   1      0
19999   8   6  44   8   9  43   5   6      0

[10000 rows x 9 columns]
p3    99
p6    99
dtype: int64


In [48]:
neg_process = df_raw.iloc[10000:,:].copy()
print(neg_process)

for i in range(10000):
    if neg_process.iloc[i,0] > df_max[0]:
        neg_process.iloc[i,0]  = int(neg_process.iloc[i,2] / df_max[0])
    if neg_process.iloc[i,1] > df_max[1]:
        neg_process.iloc[i,1]  = int(neg_process.iloc[i,5] / df_max[1])

       p1  p2  p3  p4  p5  p6  p7  p8  label
10000   1   0  62   0   1  81   0   0      0
10001   1   1  52   1   1  18   0   0      0
10002   1   1  71   0   1  20   0   0      0
10003   1   1  37   0   1  61   1   1      0
10004   1   0  47   1   0  12   0   1      0
...    ..  ..  ..  ..  ..  ..  ..  ..    ...
19995   9   8  28   0   4  91   2   7      0
19996   1   9  98   2   9  85   9   7      0
19997   0   8  49   3   1  17   6   4      0
19998   1   2  35   9   7  55   3   1      0
19999   8   6  44   8   9  43   5   6      0

[10000 rows x 9 columns]


In [49]:
neg_process.drop_duplicates(inplace=True)
print(neg_process)
print(np.amax(neg_process))

       p1  p2  p3  p4  p5  p6  p7  p8  label
10000   1   0  62   0   1  81   0   0      0
10001   1   1  52   1   1  18   0   0      0
10002   1   1  71   0   1  20   0   0      0
10003   1   1  37   0   1  61   1   1      0
10004   1   0  47   1   0  12   0   1      0
...    ..  ..  ..  ..  ..  ..  ..  ..    ...
19995   9   8  28   0   4  91   2   7      0
19996   1   9  98   2   9  85   9   7      0
19997   0   8  49   3   1  17   6   4      0
19998   1   2  35   9   7  55   3   1      0
19999   8   6  44   8   9  43   5   6      0

[10000 rows x 9 columns]
p1        9
p2        9
p3       99
p4        9
p5        9
p6       99
p7        9
p8        9
label     0
dtype: int64


In [50]:
df_process = pd.concat([df_raw.iloc[:10000,:], neg_process])
print(df_process)

       p1  p2  p3  p4  p5  p6  p7  p8  label
0       0   1   0   0   0   0   0   0      1
1       1   0   0   0   0   0   0   0      1
2       0   1   1   0   0   0   0   0      1
3       0   0   0   1   0   0   0   0      1
4       1   0   1   0   0   0   0   0      1
...    ..  ..  ..  ..  ..  ..  ..  ..    ...
19995   9   8  28   0   4  91   2   7      0
19996   1   9  98   2   9  85   9   7      0
19997   0   8  49   3   1  17   6   4      0
19998   1   2  35   9   7  55   3   1      0
19999   8   6  44   8   9  43   5   6      0

[20000 rows x 9 columns]


In [51]:
x_data = df_process.iloc[:,:-1]
y_data = df_process.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [52]:
x_train_input = pd.concat([x_train.iloc[:,:], y_train],axis=1)
x_test_input = pd.concat([x_test.iloc[:,:],y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [53]:
x_train_input.head(10)

,p1,p2,p3,p4,p5,p6,p7,p8,label
8118,1,0,16,0,0,28,0,0,1
10171,1,1,69,1,1,31,0,0,0
9399,0,1,4,0,0,37,0,0,1
12669,0,1,51,0,1,41,0,1,0
18809,4,1,84,0,5,14,4,1,0
13869,1,1,36,0,1,30,1,1,0
3320,0,1,14,0,0,16,0,0,1
14689,1,0,37,0,1,19,1,1,0
13087,0,0,39,1,0,15,1,1,0
15992,0,1,66,1,1,20,0,1,0


In [54]:
x_test_input.head(10)

,p1,p2,p3,p4,p5,p6,p7,p8,label
14356,1,0,80,1,1,98,0,0,0
3439,0,0,27,1,0,10,0,0,1
12153,0,1,86,0,1,49,1,1,0
15029,0,1,90,1,1,7,0,0,0
18549,9,2,46,4,3,37,0,5,0
15762,1,0,39,0,0,9,1,0,0
12313,1,0,61,1,1,31,0,0,0
16034,0,1,69,1,0,42,0,1,0
11496,0,0,97,0,1,56,1,0,0
5653,0,0,34,0,1,14,0,0,1


In [55]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 5000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

print(X)
print(y)

       p1  p2  p3  p4  p5  p6  p7  p8
8118    1   0  16   0   0  28   0   0
10171   1   1  69   1   1  31   0   0
9399    0   1   4   0   0  37   0   0
12669   0   1  51   0   1  41   0   1
18809   4   1  84   0   5  14   4   1
...    ..  ..  ..  ..  ..  ..  ..  ..
13927   1   0  32   1   0  93   1   0
919     0   0  20   1   0   2   0   0
5699    0   0   4   0   0  28   1   0
10742   1   0  74   1   1  20   1   1
16921   0   1   6   0   1   8   1   1

[16000 rows x 8 columns]
8118     0
10171    0
9399     1
12669    0
18809    0
        ..
13927    0
919      0
5699     1
10742    0
16921    0
Name: label, Length: 16000, dtype: int64


In [56]:
pd.Series(y).value_counts()

0    12923
1     3077
Name: label, dtype: int64

# Trainning directly

In [57]:
print('Training XGboost model ...')

#import xgboost as xgb

#model = xgb.XGBClassifier()

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()


#from sklearn.neural_network import MLPClassifier

#model = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1), random_state=1,max_iter=100000)

model.fit(X, y)

print('Done')

Training XGboost model ...
Done


In [58]:
print('---- {} ----'.format('XGboost model'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test_input.iloc[:,:-1])), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, model.predict(x_test_input.iloc[:,:-1])))
print('Recall: ', recall_score(y_test, model.predict(x_test_input.iloc[:,:-1])))
print('Accuracy: ', accuracy_score(y_test, model.predict(x_test_input.iloc[:,:-1])))
print('f1_score: ', f1_score(y_test, model.predict(x_test_input.iloc[:,:-1])))

---- XGboost model ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive       1923.0          0.0 
None

Precision:  0.0
Recall:  0.0
Accuracy:  0.51925
f1_score:  0.0


# Training by bagging

In [59]:
#初始化多个分类器

from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()

from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1), random_state=1,max_iter=100000)

from sklearn import svm
model4 = svm.LinearSVC()

from sklearn.naive_bayes import MultinomialNB
model5 = MultinomialNB()

from sklearn.ensemble import RandomForestClassifier
model6 = RandomForestClassifier(n_estimators = 50,n_jobs = -1)

import xgboost as xgb
model7 = xgb.XGBClassifier()


In [60]:
model_list = [model1, model2, model3, model4, model5, model6, model7]
f1 = np.zeros([len(model_list)],dtype=np.float32)

In [61]:
for i,j in zip(model_list,range(len(model_list))):
    model = BaggingClassifierPU(i,
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
    model.fit(X,y)
    print(f1_score(y_orig, model.predict(X)))
    f1[j] = f1_score(y_orig, model.predict(X))

print(f1)

1.0
0.8889652801650051
1.0
1.0
0.6273855598558655
0.9523315390103119
0.9898712017006377
[1.         0.8889653  1.         1.         0.62738556 0.95233154
 0.9898712 ]


In [62]:
f1_index = []
for i in range(len(f1)):
    if f1[i] >= 0.95:
        f1_index.append(i)
        
print(f1_index)
        
predict_sum = np.zeros([len(X)],dtype=np.float32)
for i in f1_index:
    model = BaggingClassifierPU(model_list[i],
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
    model.fit(X,y)
    predict_sum += model.predict(X)
print(predict_sum)

[0, 2, 3, 5, 6]
[5. 0. 5. ... 5. 0. 0.]


In [63]:
threshold = len(f1_index) / 2
print(predict_sum)

predict = np.zeros([len(y)],dtype=np.int64)

for i in range(len(X)):
    if predict_sum[i] >= threshold :
        predict[i] = 1
    if predict_sum[i] < threshold :
        predict[i] = 0


print(predict)
print(y_orig)

[5. 0. 5. ... 5. 0. 0.]
[1 0 1 ... 1 0 0]
8118     1
10171    0
9399     1
12669    0
18809    0
        ..
13927    0
919      1
5699     1
10742    0
16921    0
Name: label, Length: 16000, dtype: int64


In [64]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig,predict), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, predict))
print('Recall: ', recall_score(y_orig,predict))
print('Accuracy: ', accuracy_score(y_orig, predict))
print('f1_score: ', f1_score(y_orig, predict))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive          0.0       8077.0 
None

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0


In [65]:
print(np.argmax(f1))

best_model = BaggingClassifierPU(model_list[np.argmax(f1)],
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
print(best_model)

0
BaggingClassifierPU(base_estimator=LogisticRegression(), max_samples=3077,
                    n_estimators=50, n_jobs=-1)


In [66]:
print('Training bagging classifier...')

pu_start = time.perf_counter()

best_model.fit(X, y)
pu_end = time.perf_counter()
print('Done!')
print('Time:', pu_end - pu_start)


Training bagging classifier...
Done!
Time: 3.5928057371638715


In [67]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, best_model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, best_model.predict(X)))
print('Recall: ', recall_score(y_orig, best_model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, best_model.predict(X)))
print('f1_score: ', f1_score(y_orig, best_model.predict(X)))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive          0.0       8077.0 
None

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0


In [68]:
#print wrong predictions
y_pre = model.predict(X)
y_orig_index = y_orig.index.tolist()

FN_index = []
FT_index = []

for i in range(len(y_orig)):
    if y_orig.iloc[i] == 1 and y_pre[i] == 0 :
        FN_index.append(y_orig_index[i])
    if y_orig.iloc[i] == 0 and y_pre[i] == 1 :
        FT_index.append(y_orig_index[i])
        
print("False Negtive:")
print(df_process.loc[FN_index])
print("False Positive:")
print(df_process.loc[FT_index])

False Negtive:
      p1  p2  p3  p4  p5  p6  p7  p8  label
9281   1   0   3   0   0  37   0   0      1
6742   0   0  16   1   0  25   0   0      1
630    1   0   9   0   0   5   0   0      1
9545   0   0  63   1   0   8   0   0      1
455    1   0  10   0   0   3   0   0      1
...   ..  ..  ..  ..  ..  ..  ..  ..    ...
6572   0   0  65   0   0   0   1   0      1
6871   0   1  68   0   0   0   0   0      1
9292   0   0  72   1   0   3   0   0      1
4718   0   0   9   1   0  23   0   0      1
8225   0   0  17   1   0  28   0   0      1

[163 rows x 9 columns]
False Positive:
       p1  p2  p3  p4  p5  p6  p7  p8  label
15116   0   1   2   1   0  36   0   0      0


In [69]:
#test data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, best_model.predict(x_test_input.iloc[:,:-1])), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))
print('Recall: ', recall_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))
print('Accuracy: ', accuracy_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))
print('f1_score: ', f1_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive          0.0       1923.0 
None

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0


In [70]:
#print wrong predictions
y_test_pre = model.predict(x_test_input.iloc[:,:-1])
y_test_index = y_test.index.tolist()

FN_test_index = []
FT_test_index = []

for i in range(len(y_test)):
    if y_test.iloc[i] == 1 and y_test_pre[i] == 0 :
        FN_test_index.append(y_test_index[i])
    if y_test.iloc[i] == 0 and y_test_pre[i] == 1 :
        FT_test_index.append(y_test_index[i])

print("False Negtive:")
print(df_process.loc[FN_test_index])
print("False Positive:")
print(df_process.loc[FT_test_index])

False Negtive:
      p1  p2  p3  p4  p5  p6  p7  p8  label
6177   0   0   0   0   0  31   0   1      1
8569   1   0  14   0   0  30   0   0      1
9244   0   0  27   0   0  25   1   0      1
215    0   0   6   0   0   2   0   1      1
6473   0   0  63   1   0   1   0   0      1
5088   0   0  51   0   0   3   1   0      1
8366   0   0  74   1   0   0   0   0      1
2383   0   0  26   0   0   6   0   1      1
9938   0   0  44   0   1  19   0   0      1
9720   1   0  29   0   0  25   0   0      1
6084   0   0  63   1   0   0   0   0      1
7480   1   0   9   0   0  30   0   0      1
94     0   0   6   0   0   0   0   1      1
6755   0   0   9   0   0  28   0   1      1
7924   0   0  72   1   0   0   0   0      1
6468   0   0   0   0   1  33   0   0      1
6276   0   0  64   1   0   0   0   0      1
9884   1   0  81   0   0   0   0   0      1
9596   0   0  29   1   0  25   0   0      1
9961   1   0  30   0   0  25   0   0      1
827    1   0  10   0   0   6   0   0      1
7489   1   0   3 

In [71]:
orig_data = df_process.iloc[:,:-1].copy()
orig_label = model.predict(orig_data)

print(orig_data)
print(orig_label)

       p1  p2  p3  p4  p5  p6  p7  p8
0       0   1   0   0   0   0   0   0
1       1   0   0   0   0   0   0   0
2       0   1   1   0   0   0   0   0
3       0   0   0   1   0   0   0   0
4       1   0   1   0   0   0   0   0
...    ..  ..  ..  ..  ..  ..  ..  ..
19995   9   8  28   0   4  91   2   7
19996   1   9  98   2   9  85   9   7
19997   0   8  49   3   1  17   6   4
19998   1   2  35   9   7  55   3   1
19999   8   6  44   8   9  43   5   6

[20000 rows x 8 columns]
[1 1 1 ... 0 0 0]


In [72]:
import xgboost as xgb
model = BaggingClassifierPU(xgb.XGBClassifier(),
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
print(model)
model.fit(orig_data, orig_label)

BaggingClassifierPU(base_estimator=XGBClassifier(base_score=None, booster=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 gamma=None, gpu_id=None,
                                                 importance_type='gain',
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=1

BaggingClassifierPU(base_estimator=XGBClassifier(base_score=None, booster=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 gamma=None, gpu_id=None,
                                                 importance_type='gain',
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=1

In [73]:
#verification
import random

place_max = max(np.amax(df_process.iloc[:10000, [2, 5]]))  #w_dependence
name1 = ['min', 'max', 'Precision', 'Recall', 'Accuracy', 'f1_score','time']
test = pd.DataFrame(columns=name1)

for i in range(1, 100):
    pu_start = time.perf_counter()
        
    pos_list = np.random.randint(low=100, high=9999, size=2000).tolist()
    neg_list = np.random.randint(low=10000, high=15000, size=2000).tolist()
    all_list = pos_list + neg_list
    
    flag1 = int((0.1 * (i - 1)) * place_max)
    flag2 = int((0.1 * i) * place_max)
    print(flag2)
    
    temp_data = df_process.iloc[all_list, : -1].copy()
    temp_label = df_process.iloc[all_list, -1].copy()
    
    temp_data.iloc[:, 2] = temp_data.iloc[:, 2] + random.randint((df_max[0] + flag1 + 1), ((df_max[0] + flag2 + 1)))
    temp_data.iloc[:, 5] = temp_data.iloc[:, 5] + random.randint((df_max[1] + flag1 + 1), ((df_max[1] + flag2 + 1)))

    
    print('======')
    print(temp_data.iloc[:, :])

    temp_max = max(np.amax(temp_data.iloc[:, [2, 5]]))
    temp_min = min(np.amin(temp_data.iloc[:, [2, 5]]))

    print('---- {} ----'.format(i))
    print('Precision: ', precision_score(temp_label, model.predict(temp_data)))
    print('Recall: ', recall_score(temp_label, model.predict(temp_data)))
    print('Accuracy: ', accuracy_score(temp_label, model.predict(temp_data)))
    print('f1_score: ', f1_score(temp_label, model.predict(temp_data)))

    pu_end = time.perf_counter()
    
    test.loc[i] = [  temp_min, temp_max,
                        precision_score(temp_label, model.predict(temp_data)),
                        recall_score(temp_label, model.predict(temp_data)),
                        accuracy_score(temp_label, model.predict(temp_data)),
                        f1_score(temp_label, model.predict(temp_data)),
                         (pu_end - pu_start)]
    
    orig_data = pd.concat([orig_data, temp_data], ignore_index=True)
    orig_label = pd.Series(orig_label.tolist() + model.predict(temp_data).tolist())

    model.fit(orig_data,orig_label)

test.to_csv('../result/w_dependence_unknown_processdata_result.csv')

8
       p1  p2   p3  p4  p5   p6  p7  p8
5702    0   0   91   0   0   74   1   0
7124    0   0  123   1   0   62   0   0
9020    1   0  109   0   0   73   0   0
4885    0   1  103   0   0   66   0   0
4608    1   0  117   0   0   58   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
14441   1   1  169   1   0  126   1   1
12678   1   1  157   1   0   90   1   1
13260   1   1  120   0   0   56   1   1
11026   1   0  138   0   1  102   0   1
11511   1   0  107   1   0  142   1   1

[4000 rows x 8 columns]
---- 1 ----
Precision:  0.0
Recall:  0.0
Accuracy:  0.5
f1_score:  0.0
16
       p1  p2   p3  p4  p5   p6  p7  p8
4649    0   0   96   0   0   80   1   0
8097    1   0  126   0   0   74   0   0
5912    0   0  144   1   0   60   0   0
3049    0   1  106   0   0   70   0   0
946     0   0   98   1   0   64   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
11862   1   0  100   0   0  126   1   1
13525   0   1  142   0   0  152   1   0
10154   0   1  141   1   0   54   1   1
12167   0   1  177 

KeyboardInterrupt: 